# Урок 2: Основы LoRA — математика и реализация

**Ключевые концепции:**
- Разложение матрицы: `W + ΔW = W + A × B`, где `A ∈ ℝ^(d×r)`, `B ∈ ℝ^(r×k)`, `r << d,k`
- **Rank (r)** — главный гиперпараметр (обычно 4-64)
- **Alpha (α)** — масштабирование адаптации

In [ ]:
import torch
import torch.nn as nn

class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=8, alpha=16):
        super().__init__()
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        # Frozen исходный вес
        self.weight = nn.Parameter(torch.randn(out_features, in_features), requires_grad=False)
        
        # LoRA адаптация
        self.lora_A = nn.Parameter(torch.randn(rank, in_features))
        self.lora_B = nn.Parameter(torch.randn(out_features, rank))
        
    def forward(self, x):
        base_output = x @ self.weight.T
        lora_output = (x @ self.lora_A.T) @ self.lora_B.T * self.scaling
        return base_output + lora_output

# Тест
layer = LoRALayer(128, 64, rank=8)
x = torch.randn(10, 128)
output = layer(x)
print(f"Вход: {x.shape}, Выход: {output.shape}")
print(f"Обучаемых параметров: {sum(p.numel() for p in layer.parameters() if p.requires_grad)}")
print(f"Всего параметров: {sum(p.numel() for p in layer.parameters())}")